## 历史以及基操
正则表达式 -> 上下文无关语法  -> 统计学派+规则学派 -> 隐马尔可夫  -> 循环神经网络  -> 深度学习有效性  -> Word2Vec（词向量）

### 歧义

- 词法分析歧义、语法分析歧义、语义分析歧义、NLP实际应用歧义（如语音识别）     
口语处理、分词处理、新词语产生、上下文知识、实体词   
***
- 1.词汇表， 字符表
- 2.句子或词语概率分布等级并由此引出字后面某字的概率（贝叶斯链式法则）

### 语言模型

- 一阶马尔科夫模型（当前随机变量取值只与上一时刻有关）
    - 句子中某个字只与前面一个字或词有关
- 二阶马尔科夫模型（当前随机变量取值只与上两时刻有关）
    - 句子中某个字只与前面2个字或词有关
- 可变长序列（句子长度是随机），定义停止状态符号和开始状态符号
- 三元语言模型 (N^3)

```
句子： **我是中国人<s>
模型： p(我是中国人<s>)=q(我|*，*)q(是|我，*)...q(<s>|国，人)
```
- 极大似然估计             
参数： q(w|u,v)                  
MLE: q(w|u,v) = Count(w,u,v) / Count(u,v) -> q(人|国,中) = Count(中,国,人) / Count(国,人)            
- 语言模型评价：迷惑度      
    - 测试集： m个句子$s_1,s_2,...,s_m$             
    - log概率：    大
        - $log \prod_{i=0}^mp(s_i) = \sum_{i=1}^mlog(p(s_i))$           
    - perplexity： 小
        - $Perplexity = 2^{-l}$   
        - $l = \frac{1}{M}\sum_{i=1}^mlog(p(s_i))$ _M是测试集总词量_